In [1]:

import datetime
import time

import random
import math
import matplotlib.pyplot as plt
import numpy as np

from tqdm import tqdm_notebook as tqdm 

import os, shutil
from zipfile import ZipFile

import skimage
from skimage.io import imread,imsave
from skimage import morphology
from skimage.filters import threshold_otsu
from skimage import img_as_uint


from digest_patchExtractor import DualPatchExtractor
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
from models_pytorch import DualEncoding_U_Net,init_weights
from DigestPath_Dataset import DataSet,ToTensor,Scale,Color

from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F

import pandas as pd 

# from preprocess import fuse_roi,generate_train_mask
# from prediction import whole_img_pred,post_process

/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [ ]:
map1_dir_train='/home/vahadaneabhi01/datalab/training-assets/R_medical/atheeth/digest_new/Colonoscopy_tissue_segment_dataset/map_1_train/'
map2_dir_train='/home/vahadaneabhi01/datalab/training-assets/R_medical/atheeth/digest_new/Colonoscopy_tissue_segment_dataset/map_2_train/'
mask_dir_train='/home/vahadaneabhi01/datalab/training-assets/R_medical/atheeth/digest_new/Colonoscopy_tissue_segment_dataset/mask_train/'
map1_patch_dir_train='/home/vahadaneabhi01/datalab/training-assets/R_medical/atheeth/digest_new/Colonoscopy_tissue_segment_dataset/map_1_patches_train/'
map2_patch_dir_train='/home/vahadaneabhi01/datalab/training-assets/R_medical/atheeth/digest_new/Colonoscopy_tissue_segment_dataset/map_2_patches_train/'
mask_patch_dir_train='/home/vahadaneabhi01/datalab/training-assets/R_medical/atheeth/digest_new/Colonoscopy_tissue_segment_dataset/mask_patches_train/'

map1_dir_test='/home/vahadaneabhi01/datalab/training-assets/R_medical/atheeth/digest_new/Colonoscopy_tissue_segment_dataset/map_1_test/'
map2_dir_test='/home/vahadaneabhi01/datalab/training-assets/R_medical/atheeth/digest_new/Colonoscopy_tissue_segment_dataset/map_2_test/'
mask_dir_test='/home/vahadaneabhi01/datalab/training-assets/R_medical/atheeth/digest_new/Colonoscopy_tissue_segment_dataset/mask_test/'
map1_patch_dir_test='/home/vahadaneabhi01/datalab/training-assets/R_medical/atheeth/digest_new/Colonoscopy_tissue_segment_dataset/map_1_patches_test/'
map2_patch_dir_test='/home/vahadaneabhi01/datalab/training-assets/R_medical/atheeth/digest_new/Colonoscopy_tissue_segment_dataset/map_2_patches_test/'
mask_patch_dir_test='/home/vahadaneabhi01/datalab/training-assets/R_medical/atheeth/digest_new/Colonoscopy_tissue_segment_dataset/mask_patches_test/'

dir_list=[map1_dir_train,map2_dir_train,mask_dir_train,map1_patch_dir_train,\
          map2_patch_dir_train,mask_patch_dir_train,map1_dir_test,map2_dir_test,\
          mask_dir_test,map1_patch_dir_test,map2_patch_dir_test,mask_patch_dir_test]
for directory in dir_list:
    if not os.path.exists(directory):
        os.system("mkdir {}".format(directory))
        print('made directory')

In [ ]:

map_1_dir='Colonoscopy_tissue_segment_dataset/map_1'
map_2_dir='Colonoscopy_tissue_segment_dataset/map_2'
mask_dir='Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_mask'

all_img_list=os.listdir(map_1_dir)
train_list=random.sample(all_img_list, 200)
test_list=list(set(all_img_list)-set(train_list))
# for train_img in tqdm(train_list):
#     mask_img_name=train_img.split('.')[0]+'_mask.jpg'
#     shutil.copy(os.path.join(map_1_dir,train_img),os.path.join(map1_dir_train,train_img))
#     shutil.copy(os.path.join(map_2_dir,train_img),os.path.join(map2_dir_train,train_img))
#     shutil.copy(os.path.join(mask_dir,mask_img_name),os.path.join(mask_dir_train,mask_img_name))
    
# for test_img in tqdm(test_list):
#     test_mask_img_name=test_img.split('.')[0]+'_mask.jpg'
#     shutil.copy(os.path.join(map_1_dir,test_img),os.path.join(map1_dir_test,test_img))
#     shutil.copy(os.path.join(map_2_dir,test_img),os.path.join(map2_dir_test,test_img))
#     shutil.copy(os.path.join(mask_dir,test_mask_img_name),os.path.join(mask_dir_test,test_mask_img_name))

In [ ]:

# train_extractor=DualPatchExtractor(map1_dir_train,map2_dir_train,mask_dir_train,map1_patch_dir_train,map2_patch_dir_train\
#                                ,mask_patch_dir_train,overlap=False,progress_bar=True)

# train_extractor.extract_patches()

# test_extractor=DualPatchExtractor(map1_dir_test,map2_dir_test,mask_dir_test,map1_patch_dir_test,map2_patch_dir_test\
#                                ,mask_patch_dir_test,overlap=False,progress_bar=True)

# test_extractor.extract_patches()

In [ ]:
batch_size_train=4
batch_size_test=4
transform=torchvision.transforms.Compose([Scale(),ToTensor()])    



train_dataset=DataSet(map1_patch_dir_train,map2_patch_dir_train,mask_patch_dir_train,transform=transform)
train_loader=DataLoader(train_dataset,batch_size=batch_size_train,num_workers=0,shuffle=True)
print(train_dataset.__len__()," Train samples")

test_dataset=DataSet(map1_patch_dir_test,map2_patch_dir_test,mask_patch_dir_test,transform=transform)
test_loader=DataLoader(test_dataset,batch_size=batch_size_test,num_workers=0,shuffle=False)
print(test_dataset.__len__()," Test samples")


In [ ]:
    
def dice_metric(y_pred,y_true):
    smooth = 1
    num = y_true.size(0)
    m1 = y_pred.view(num, -1)
    m2 = y_true.view(num, -1)
    
        
    intersection = (m1* m2)

    score = 2. * (intersection.sum(1) + smooth) / (m1.sum(1) + m2.sum(1) + smooth)
    score = score.sum() / num
        
    return score
    

In [ ]:
def visualize_loader(loader,index):
    for i,sample in enumerate(loader):
        #print(sample['image'].shape)
        if i==1:
            map1=(sample['map1'][index]).numpy()
            
            
            #image=np.zeros(image_i.shape,dtype=np.uint8)
            #image[np.where(image_i!=0)]=255

            map2=(sample['map2'][index]).numpy()
            
        
            mask=(sample['mask'][index]).numpy()
        
           
    
            map1=map1.transpose(1,2,0)
            map2=map2.transpose(1,2,0)
            mask=np.squeeze(mask.transpose(1,2,0),axis=2)
            print(map1.shape,np.amax(map1))
        
            fig=plt.figure()
            plt.imshow(map1)
            fig2=plt.figure()
            plt.imshow(map2)
            fig3=plt.figure()
            plt.imshow(mask)
            break
visualize_loader(test_loader,2)

In [ ]:
model=DualEncoding_U_Net()
model_start_date=datetime.datetime.now().strftime("%Y_%m_%d")
BEST_MODEL_PATH=os.path.join(os.getcwd(),'model_{}'.format(model_start_date))
if not os.path.exists(BEST_MODEL_PATH):
    os.mkdir(BEST_MODEL_PATH)
    print('model_{} dir has been made'.format(model_start_date))
print("Model's state_dict:")
writer = SummaryWriter('model_{}/experiment_{}'.format(model_start_date,1))
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())


# writer.close()

In [ ]:
print(torch.cuda.get_device_name(torch.cuda.current_device()))
optimizer_selected='adam'
batchsize=4
no_steps=train_dataset.__len__()//batchsize
restart_epochs=8
num_epochs=10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#criterion = SoftDiceLoss()#
criterion=nn.BCELoss()

history={'train_loss':[],'test_loss':[],'train_dice':[],'test_dice':[]}
model = model.to(device)
if optimizer_selected=='adam':
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-03, betas=(0.9, 0.98))#,weight_decay=0.02)
else:
    optimizer = torch.optim.SGD(model.parameters(),lr=1e-03, momentum=0.8,nesterov=True)

scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, restart_epochs*no_steps,\
                                                     eta_min=1e-012, last_epoch=-1)


best_val=0
for epoch in range(num_epochs):
    
    print("Learning Rate : {}".format(optimizer.state_dict()['param_groups'][-1]['lr']))
    # loop over the dataset multiple times
    
    run_avg_train_loss=0
    run_avg_train_dice=0
    
   
    
    run_avg_test_loss=0
    run_avg_test_dice=0
    
    for mode in ['train','eval']:
     
        if mode == 'train':
            
            model.train()
            loop=tqdm(train_loader)
            
            for i, sample_batched in (enumerate(loop)):
                loop.set_description('Epoch {}/{}'.format(epoch + 1, num_epochs))
                
                #Clear Gradients
                optimizer.zero_grad()
                
                # get the inputs; data is a list of [dapi, nuclei, boundary]
                map1_batch, map2_batch,mask_batch = sample_batched['map1'],\
                sample_batched['map2'],sample_batched['mask']
                
                map1_batch, map2_batch,mask_batch = map1_batch.to(device, dtype = torch.float)\
                ,map2_batch.to(device, dtype = torch.float),mask_batch.to(device,dtype=torch.float)

                # forward + backward + optimize
                outputs = torch.sigmoid(model(map1_batch,map2_batch))
                
                loss = criterion(outputs, mask_batch)
                dice_score=dice_metric(outputs,mask_batch)
                run_avg_train_loss=(run_avg_train_loss*(0.9))+loss.detach().item()*0.1
                run_avg_train_dice=(run_avg_train_dice*(0.9))+dice_score.detach().item()*0.1
               
                if i%100==99:
                    
                
                    
                    img_tensor=torch.cat((outputs.detach().cpu(),mask_batch.detach().cpu()),axis=0)
                    
                    
            
                    img_grid2 = torchvision.utils.make_grid(img_tensor*255,nrow=batch_size_train,padding=100)
                    torchvision.utils.save_image(img_grid2,os.path.join(BEST_MODEL_PATH,\
                                                                        'train_iter_{}.png'\
                                                                        .format(epoch*len(train_loader)+i+1)))
                    writer.add_image('TRAIN_ITER_{}'.format(epoch * len(train_loader) + i+1), img_grid2)
                    
                    
                    writer.add_scalar('Training dice score nuclei',
                            run_avg_train_dice,
                            epoch * len(train_loader) + i+1)
                
                    writer.add_scalar('Training Loss',
                            run_avg_train_loss,
                            epoch * len(train_loader) + i+1)
                    
                loss.backward()
                optimizer.step()
                
                scheduler.step()
                
                
                loop.set_postfix(loss=run_avg_train_loss,dice_score=run_avg_train_dice)
               
            history['train_loss'].append(run_avg_train_loss)
            history['train_dice'].append(run_avg_train_dice)
                
                 
                    
        elif mode =='eval':
            #Clear Gradients
            optimizer.zero_grad()
            samples_test=len(test_loader)
            model.eval()
            val_loss=0
            test_agg=0
            for j, test_sample in enumerate(test_loader):

                test_map1_batch, test_map2_mask_batch,test_mask_batch = test_sample['map1']\
                , test_sample['map2'],test_sample['mask']
                
                test_map1_batch, test_map2_mask_batch,test_mask_batch = test_map1_batch.to(device, dtype = torch.float),\
                test_map2_mask_batch.to(device, dtype = torch.float),test_mask_batch.to(device, dtype = torch.float)
                test_outputs = torch.sigmoid(model(test_map1_batch,test_map2_batch))
                
                test_loss = criterion(test_outputs, test_mask_batch)
                #final_test_loss+=test_loss.detach().item()
                test_dice=dice_metric(test_outputs,test_mask_batch)
                #final_test_dice+=test_dice
                run_avg_test_loss=(run_avg_test_loss*(0.9))+test_loss.detach().item()*0.1
                run_avg_test_dice=(run_avg_test_dice*(0.9))+test_dice.detach().item()*0.1
               
                if j%100==99:
                    
                    img_tensor_test=torch.cat((test_outputs.detach().cpu(),test_mask_batch.detach().cpu()),axis=0)
                    
                    
                    img_grid = torchvision.utils.make_grid(img_tensor_test*255,nrow=batch_size_test,padding=10)
                    torchvision.utils.save_image(img_grid,os.path.join(BEST_MODEL_PATH,\
                                                                        'test_iter_{}.png'\
                                                                        .format(epoch*len(test_loader)+j+1)))
                    writer.add_image('TEST_ITER_{}'.format(epoch * len(test_loader) + j+1), img_grid)
                    writer.add_scalar('Testing dice score ',\
                                      run_avg_test_dice,epoch * len(test_loader) + j+1)
                    
                    writer.add_scalar('Testing Loss',\
                                      run_avg_test_loss,epoch * len(test_loader) + j+1)
                
            print("test_loss: {}\ntest_dice :{}"\
                  .format(run_avg_test_loss,run_avg_test_dice))
            history['test_loss'].append(run_avg_test_loss)
            history['test_dice'].append(run_avg_test_dice)
            if run_avg_test_dice>best_val:
                best_val=run_avg_test_dice
                torch.save(model.state_dict(), BEST_MODEL_PATH+'/model_optim.pth')
                print("saved model with test dice score: {}".format(best_val))

        
#             print("val_loss {}".format(val_loss/samples_test))
torch.save(model.state_dict(), BEST_MODEL_PATH+'/model_final.pth')
print('Finished Training')



